# Simple Pixel-Level Heat Vulnerability Analysis

Extract individual pixel data from WorldPop for all cities in a selected country. Each row in the output represents a single WorldPop pixel with heat vulnerability calculations.

In [1]:
import ee
import pandas as pd
import numpy as np
import time
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Initialize GEE
ee.Initialize(project='tl-cities')
print('✅ GEE initialized for pixel-level analysis')

✅ GEE initialized for pixel-level analysis


In [2]:
# Load datasets
allCities = ee.FeatureCollection('projects/tl-cities/assets/GHS_UCDB_THEME_HAZARD_RISK_GLOBE_R2024A')
worldpopCollection = ee.ImageCollection('WorldPop/GP/100m/pop_age_sex')

# Get WorldPop data for 2020
worldpop_2020 = worldpopCollection.filter(ee.Filter.eq('year', 2020)).mosaic()

print(f"Total cities globally: {allCities.size().getInfo()}")
print(f"WorldPop 2020 bands: {len(worldpop_2020.bandNames().getInfo())}")
print('✅ Datasets loaded')

Total cities globally: 11422
WorldPop 2020 bands: 37
✅ Datasets loaded


## Country Selection and City Filtering

In [3]:
def get_available_countries():
    """Get list of all countries with cities"""
    print('🌍 Loading available countries...')
    countries = allCities.aggregate_array('GC_CNT_GAD').distinct().sort().getInfo()
    # Filter out None/null values
    countries = [c for c in countries if c and c != 'Unknown']
    print(f'✅ Found {len(countries)} countries with city data')
    return countries

def get_cities_for_country(country_name, min_population=100000, max_cities=50):
    """Get cities for a specific country with population filter"""
    print(f'🔍 Getting cities for {country_name}...')
    
    # Filter cities by country and population
    country_cities = (allCities
        .filter(ee.Filter.eq('GC_CNT_GAD', country_name))
        .filter(ee.Filter.gt('GC_POP_TOT', min_population))
        .sort('GC_POP_TOT', False)
        .limit(max_cities)
    )
    
    city_count = country_cities.size().getInfo()
    print(f'✅ Found {city_count} cities in {country_name} (pop > {min_population:,})')
    
    # Show top 10 cities
    if city_count > 0:
        sample = country_cities.limit(10).getInfo()
        print(f'\n📊 Top cities:')
        for i, city in enumerate(sample['features']):
            props = city['properties']
            name = props.get('GC_UCN_MAI', 'Unknown')
            pop = props.get('GC_POP_TOT', 0)
            print(f'  {i+1:2d}. {name:30s} - {pop:>12,.0f} people')
    
    return country_cities

# Load available countries
available_countries = get_available_countries()
print(f'\n🌎 Sample countries: {available_countries[:10]}')

🌍 Loading available countries...
✅ Found 191 countries with city data

🌎 Sample countries: ['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan']


## Pixel-Level Data Extraction Functions

In [4]:
def extract_pixel_level_data_for_country(country_name, max_cities=20, max_pixels_per_city=10000):
    """Extract pixel-level data for all cities in a country"""
    
    print(f'🔬 Extracting pixel-level data for {country_name}')
    print(f'   Max cities: {max_cities}')
    print(f'   Max pixels per city: {max_pixels_per_city:,}')
    print('='*60)
    
    try:
        # Get cities for the country
        cities = get_cities_for_country(country_name, max_cities=max_cities)
        city_list = cities.getInfo()['features']
        
        if not city_list:
            print('❌ No cities found for this country')
            return None
        
        print(f'\n📊 Processing {len(city_list)} cities...')
        
        # Define age bands for vulnerability calculation
        under_5_bands = ['M_0', 'M_1', 'F_0', 'F_1']  # 0-4 years
        over_65_bands = ['M_65', 'F_65', 'M_70', 'F_70', 'M_75', 'F_75', 'M_80', 'F_80']  # 65+ years
        
        # All age bands for total population
        all_age_bands = [
            'M_0', 'M_1', 'M_5', 'M_10', 'M_15', 'M_20', 'M_25', 'M_30', 'M_35', 'M_40', 
            'M_45', 'M_50', 'M_55', 'M_60', 'M_65', 'M_70', 'M_75', 'M_80',
            'F_0', 'F_1', 'F_5', 'F_10', 'F_15', 'F_20', 'F_25', 'F_30', 'F_35', 'F_40',
            'F_45', 'F_50', 'F_55', 'F_60', 'F_65', 'F_70', 'F_75', 'F_80'
        ]
        
        # Create population sum images
        under_5_pop = worldpop_2020.select(under_5_bands).reduce(ee.Reducer.sum())
        over_65_pop = worldpop_2020.select(over_65_bands).reduce(ee.Reducer.sum())
        total_pop = worldpop_2020.select(all_age_bands).reduce(ee.Reducer.sum())
        
        # Create vulnerability ratio image: (under_5 + over_65) / total * 100
        vulnerable_pop = under_5_pop.add(over_65_pop)
        vulnerability_ratio = vulnerable_pop.divide(total_pop.add(0.001)).multiply(100)
        
        # Combine all data into single image
        combined_image = ee.Image.cat([
            under_5_pop,
            over_65_pop, 
            total_pop,
            vulnerability_ratio
        ]).rename(['under_5_pop', 'over_65_pop', 'total_pop', 'vulnerability_ratio'])
        
        all_pixel_data = []
        
        for i, city_feature in enumerate(city_list):
            city_props = city_feature['properties']
            city_name = city_props.get('GC_UCN_MAI', 'Unknown')
            ghs_population = city_props.get('GC_POP_TOT', 0)
            
            print(f'\n🏙️ Processing city {i+1}/{len(city_list)}: {city_name}')
            print(f'   GHS Population: {ghs_population:,.0f}')
            
            try:
                # Get city geometry
                city_geometry = ee.Geometry(city_feature['geometry'])
                
                # Extract pixel coordinates and values within city boundary
                # Use sample to get pixel locations and values
                pixel_data = combined_image.sample(
                    region=city_geometry,
                    scale=90,  # WorldPop resolution
                    numPixels=max_pixels_per_city,
                    seed=42,  # For reproducible sampling
                    geometries=True  # Include pixel coordinates
                ).getInfo()
                
                if not pixel_data['features']:
                    print(f'   ⚠️ No pixels found for {city_name}')
                    continue
                
                print(f'   📊 Extracted {len(pixel_data["features"])} pixels')
                
                # Calculate city-wide WorldPop totals for reference
                city_totals = combined_image.reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=city_geometry,
                    scale=90,
                    maxPixels=1e8
                ).getInfo()
                
                city_worldpop_total = city_totals.get('total_pop', 0)
                city_under_5_total = city_totals.get('under_5_pop', 0)
                city_over_65_total = city_totals.get('over_65_pop', 0)
                total_pixels_in_city = len(pixel_data['features'])
                
                print(f'   📈 City totals: {city_worldpop_total:,.0f} total, {city_under_5_total:,.0f} under-5, {city_over_65_total:,.0f} over-65')
                
                # Process each pixel
                for pixel in pixel_data['features']:
                    pixel_props = pixel['properties']
                    
                    # Get pixel coordinates (optional - for spatial analysis)
                    coords = pixel['geometry']['coordinates']
                    lon, lat = coords[0], coords[1]
                    
                    # Extract population values
                    under_5 = pixel_props.get('under_5_pop', 0) or 0
                    over_65 = pixel_props.get('over_65_pop', 0) or 0
                    total = pixel_props.get('total_pop', 0) or 0
                    vuln_ratio = pixel_props.get('vulnerability_ratio', 0) or 0
                    
                    # Create pixel record
                    pixel_record = {
                        'country_name': country_name,
                        'city_name': city_name,
                        'ghs_population': ghs_population,
                        'sum_worldpop_population': city_worldpop_total,
                        'no_pixels_in_city': total_pixels_in_city,
                        'sum_population_under_5': city_under_5_total,
                        'sum_population_over_65': city_over_65_total,
                        'total_population': city_worldpop_total,
                        'vulnerability_ratio': city_worldpop_total > 0 and (city_under_5_total + city_over_65_total) / city_worldpop_total * 100 or 0,
                        # Pixel-specific data
                        'pixel_longitude': lon,
                        'pixel_latitude': lat,
                        'pixel_under_5_pop': under_5,
                        'pixel_over_65_pop': over_65,
                        'pixel_total_pop': total,
                        'pixel_vulnerability_ratio': vuln_ratio
                    }
                    
                    all_pixel_data.append(pixel_record)
                
            except Exception as e:
                print(f'   ❌ Error processing {city_name}: {e}')
                continue
        
        if all_pixel_data:
            df = pd.DataFrame(all_pixel_data)
            
            print(f'\n🎉 Extraction completed!')
            print(f'   Total pixels extracted: {len(df):,}')
            print(f'   Cities processed: {df["city_name"].nunique()}')
            print(f'   Columns: {len(df.columns)}')
            
            return df
        else:
            print('\n❌ No pixel data extracted')
            return None
            
    except Exception as e:
        print(f'❌ Error in pixel extraction: {e}')
        return None

print('✅ Pixel extraction functions ready')

✅ Pixel extraction functions ready


## Interactive Analysis Interface

In [5]:
# Create interactive widgets
country_dropdown = widgets.Dropdown(
    options=available_countries,
    value='Brazil' if 'Brazil' in available_countries else available_countries[0],
    description='Country:',
    layout={'width': '300px'}
)

max_cities_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=50,
    step=5,
    description='Max Cities:',
    style={'description_width': 'initial'}
)

max_pixels_slider = widgets.IntSlider(
    value=5000,
    min=1000,
    max=20000,
    step=1000,
    description='Max Pixels/City:',
    style={'description_width': 'initial'}
)

extract_button = widgets.Button(
    description='🔬 Extract Pixel Data',
    button_style='primary',
    layout={'width': '200px'}
)

export_button = widgets.Button(
    description='💾 Export to CSV',
    button_style='success',
    layout={'width': '150px'},
    disabled=True
)

output_area = widgets.Output()

# Global variable to store results
pixel_results = None

def on_extract_click(button):
    global pixel_results
    
    with output_area:
        clear_output(wait=True)
        
        country = country_dropdown.value
        max_cities = max_cities_slider.value
        max_pixels = max_pixels_slider.value
        
        print(f'🚀 Starting pixel-level extraction for {country}')
        print(f'   Max cities: {max_cities}')
        print(f'   Max pixels per city: {max_pixels:,}')
        print(f'   Estimated total pixels: {max_cities * max_pixels:,}')
        
        start_time = time.time()
        
        pixel_results = extract_pixel_level_data_for_country(
            country, 
            max_cities=max_cities,
            max_pixels_per_city=max_pixels
        )
        
        if pixel_results is not None:
            processing_time = time.time() - start_time
            
            print(f'\n🎉 SUCCESS! Extraction completed in {processing_time:.1f} seconds')
            
            # Show summary statistics
            print(f'\n📊 DATASET SUMMARY:')
            print(f'   Total pixels: {len(pixel_results):,}')
            print(f'   Cities: {pixel_results["city_name"].nunique()}')
            print(f'   Columns: {len(pixel_results.columns)}')
            
            # Show sample of data
            print(f'\n📋 SAMPLE DATA (first 5 rows):')
            display(pixel_results[[
                'country_name', 'city_name', 'ghs_population', 
                'pixel_total_pop', 'pixel_under_5_pop', 'pixel_over_65_pop', 
                'pixel_vulnerability_ratio'
            ]].head())
            
            # Show vulnerability statistics
            vuln_stats = pixel_results['pixel_vulnerability_ratio']
            print(f'\n🌡️ VULNERABILITY RATIO STATISTICS:')
            print(f'   Mean: {vuln_stats.mean():.2f}%')
            print(f'   Median: {vuln_stats.median():.2f}%')
            print(f'   Min: {vuln_stats.min():.2f}%')
            print(f'   Max: {vuln_stats.max():.2f}%')
            print(f'   Std Dev: {vuln_stats.std():.2f}%')
            
            # Show top cities by pixel count
            city_pixel_counts = pixel_results.groupby('city_name').size().sort_values(ascending=False)
            print(f'\n🏙️ CITIES BY PIXEL COUNT:')
            for city, count in city_pixel_counts.head(10).items():
                avg_vuln = pixel_results[pixel_results['city_name'] == city]['pixel_vulnerability_ratio'].mean()
                print(f'   {city:30s}: {count:>6,} pixels, {avg_vuln:>6.2f}% avg vulnerability')
            
            export_button.disabled = False
            
        else:
            print('\n❌ Extraction failed')

def on_export_click(button):
    global pixel_results
    
    if pixel_results is not None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        country = country_dropdown.value.replace(' ', '_').lower()
        filename = f'/Users/martynclark/heatInsights-notebooks/data/{country}_pixel_vulnerability_{timestamp}.csv'
        
        # Round numeric columns for cleaner output
        numeric_cols = pixel_results.select_dtypes(include=[np.number]).columns
        pixel_results_rounded = pixel_results.copy()
        pixel_results_rounded[numeric_cols] = pixel_results_rounded[numeric_cols].round(4)
        
        pixel_results_rounded.to_csv(filename, index=False)
        
        print(f'✅ Results exported to: {filename}')
        print(f'   Pixels: {len(pixel_results):,}')
        print(f'   Cities: {pixel_results["city_name"].nunique()}')
        print(f'   File size: ~{len(pixel_results) * len(pixel_results.columns) * 8 / 1024 / 1024:.1f} MB')
    else:
        print('❌ No data to export - run extraction first')

extract_button.on_click(on_extract_click)
export_button.on_click(on_export_click)

print('✅ Base interface functions ready')

✅ Base interface functions ready


## Boxplot Visualization

In [6]:
import plotly.express as px
import plotly.graph_objects as go

def create_city_size_categories(df):
    """Create city size categories for aggregated analysis"""
    
    def categorize_city_size(population):
        """Categorize cities by population size"""
        if population < 500_000:
            return "Under 500K"
        elif population < 1_000_000:
            return "500K - 1M"
        elif population < 5_000_000:
            return "1M - 5M"
        elif population < 10_000_000:
            return "5M - 10M"
        else:
            return "Over 10M"
    
    # Add city size category
    df_with_categories = df.copy()
    df_with_categories['city_size_category'] = df_with_categories['ghs_population'].apply(categorize_city_size)
    
    # Define category order for consistent plotting
    category_order = ["Under 500K", "500K - 1M", "1M - 5M", "5M - 10M", "Over 10M"]
    
    return df_with_categories, category_order

def create_vulnerability_boxplot(df, metric='pixel_vulnerability_ratio', title_suffix='Vulnerability Ratio'):
    """Create boxplot showing distribution of values across individual cities"""
    
    if df is None or len(df) == 0:
        print("❌ No data available for plotting")
        return None
    
    # Get cities with enough pixels for meaningful analysis
    city_pixel_counts = df.groupby('city_name').size()
    min_pixels_threshold = 50  # Minimum pixels per city for inclusion
    cities_with_enough_pixels = city_pixel_counts[city_pixel_counts >= min_pixels_threshold].index
    
    if len(cities_with_enough_pixels) == 0:
        print(f"❌ No cities have at least {min_pixels_threshold} pixels for meaningful boxplot")
        return None
    
    # Filter data to cities with enough pixels
    filtered_df = df[df['city_name'].isin(cities_with_enough_pixels)].copy()
    
    # Sort cities by median value for better visualization
    city_medians = filtered_df.groupby('city_name')[metric].median().sort_values(ascending=True)
    city_order = city_medians.index.tolist()
    
    print(f"📊 Creating individual city boxplot for {len(city_order)} cities with ≥{min_pixels_threshold} pixels each")
    print(f"   Total pixels: {len(filtered_df):,}")
    print(f"   Metric: {metric}")
    
    # Create the boxplot
    fig = px.box(
        filtered_df, 
        x='city_name', 
        y=metric,
        title=f'Distribution of {title_suffix} Across Cities ({filtered_df["country_name"].iloc[0]})',
        labels={
            'city_name': 'City',
            metric: f'{title_suffix}' + (' (%)' if 'ratio' in metric else ''),
        },
        category_orders={'city_name': city_order}
    )
    
    # Customize the layout
    fig.update_layout(
        width=max(800, len(city_order) * 50),  # Dynamic width based on number of cities
        height=600,
        xaxis_tickangle=45,
        showlegend=False,
        title_x=0.5,
        xaxis=dict(title_font_size=14, tickfont_size=12),
        yaxis=dict(title_font_size=14, tickfont_size=12)
    )
    
    # Add summary statistics as annotations
    overall_stats = filtered_df[metric].describe()
    stats_text = (f"Overall Statistics:<br>"
                 f"Mean: {overall_stats['mean']:.2f}<br>"
                 f"Median: {overall_stats['50%']:.2f}<br>"
                 f"Std Dev: {overall_stats['std']:.2f}")
    
    fig.add_annotation(
        x=0.02, y=0.98, xref="paper", yref="paper",
        text=stats_text, showarrow=False,
        bgcolor="rgba(255,255,255,0.8)", bordercolor="gray", borderwidth=1,
        font=dict(size=10), align="left"
    )
    
    # Show the plot
    fig.show()
    
    # Print city statistics
    print(f"\n🏙️ CITY STATISTICS (sorted by median {title_suffix}):")
    city_stats = (filtered_df.groupby('city_name')[metric]
                  .agg(['count', 'mean', 'median', 'std', 'min', 'max'])
                  .round(2))
    city_stats = city_stats.reindex(city_order)
    
    for city in city_order[:15]:  # Show top 15 cities
        stats = city_stats.loc[city]
        pop = filtered_df[filtered_df['city_name'] == city]['ghs_population'].iloc[0]
        print(f"   {city[:25]:25s}: {stats['median']:>6.2f} median, {stats['mean']:>6.2f} mean, "
              f"{stats['std']:>5.2f} std, {stats['count']:>4.0f} pixels (pop: {pop:>10,.0f})")
    
    if len(city_order) > 15:
        print(f"   ... and {len(city_order)-15} more cities")
    
    return fig

def create_city_size_boxplot(df, metric='pixel_vulnerability_ratio', title_suffix='Vulnerability Ratio'):
    """Create boxplot showing distribution of values across city size categories"""
    
    if df is None or len(df) == 0:
        print("❌ No data available for plotting")
        return None
    
    print(f"📊 Creating city size-based boxplot for {title_suffix}")
    
    # Add city size categories
    df_categorized, category_order = create_city_size_categories(df)
    
    # Filter to categories with meaningful data
    min_pixels_threshold = 100
    category_pixel_counts = df_categorized.groupby('city_size_category').size()
    valid_categories = category_pixel_counts[category_pixel_counts >= min_pixels_threshold].index
    
    if len(valid_categories) == 0:
        print(f"❌ No city size categories have at least {min_pixels_threshold} pixels")
        return None
    
    df_filtered = df_categorized[df_categorized['city_size_category'].isin(valid_categories)]
    
    # Filter category order to only include categories with data
    filtered_category_order = [cat for cat in category_order if cat in valid_categories]
    
    print(f"   Total pixels: {len(df_filtered):,}")
    print(f"   City size categories: {len(filtered_category_order)}")
    
    # Create the boxplot
    fig = px.box(
        df_filtered,
        x='city_size_category',
        y=metric,
        title=f'Distribution of {title_suffix} by City Size ({df_filtered["country_name"].iloc[0]})',
        labels={
            'city_size_category': 'City Size Category',
            metric: f'{title_suffix}' + (' (%)' if 'ratio' in metric else ''),
        },
        category_orders={'city_size_category': filtered_category_order}
    )
    
    # Customize layout
    fig.update_layout(
        width=max(600, len(filtered_category_order) * 150),
        height=600,
        xaxis_tickangle=0,
        showlegend=False,
        title_x=0.5,
        xaxis=dict(title_font_size=14, tickfont_size=12),
        yaxis=dict(title_font_size=14, tickfont_size=12)
    )
    
    # Add summary statistics
    overall_stats = df_filtered[metric].describe()
    stats_text = (f"Overall Statistics:<br>"
                 f"Mean: {overall_stats['mean']:.2f}<br>"
                 f"Median: {overall_stats['50%']:.2f}<br>"
                 f"Std Dev: {overall_stats['std']:.2f}")
    
    fig.add_annotation(
        x=0.02, y=0.98, xref="paper", yref="paper",
        text=stats_text, showarrow=False,
        bgcolor="rgba(255,255,255,0.8)", bordercolor="gray", borderwidth=1,
        font=dict(size=10), align="left"
    )
    
    fig.show()
    
    # Print detailed statistics by city size
    print(f"\n🏙️ STATISTICS BY CITY SIZE CATEGORY:")
    print(f"{'Category':<12} {'Cities':<6} {'Pixels':<8} {'Mean':<7} {'Median':<7} {'Std':<6} {'Min':<6} {'Max':<6}")
    print("-" * 70)
    
    for category in filtered_category_order:
        cat_data = df_filtered[df_filtered['city_size_category'] == category]
        cities_in_category = cat_data['city_name'].nunique()
        pixel_count = len(cat_data)
        
        stats = cat_data[metric].describe()
        print(f"{category:<12} {cities_in_category:<6} {pixel_count:<8,} "
              f"{stats['mean']:<7.2f} {stats['50%']:<7.2f} {stats['std']:<6.2f} "
              f"{stats['min']:<6.2f} {stats['max']:<6.2f}")
    
    # Show example cities in each category
    print(f"\n🌆 EXAMPLE CITIES BY SIZE CATEGORY:")
    for category in filtered_category_order:
        cat_data = df_filtered[df_filtered['city_size_category'] == category]
        sample_cities = (cat_data.groupby('city_name')['ghs_population']
                       .first().sort_values(ascending=False).head(3))
        
        city_examples = []
        for city, pop in sample_cities.items():
            avg_vuln = cat_data[cat_data['city_name'] == city][metric].mean()
            city_examples.append(f"{city} ({pop:,.0f}, {avg_vuln:.1f})")
        
        print(f"   {category:<12}: {', '.join(city_examples)}")
    
    return fig

print('✅ Boxplot visualization functions ready')

✅ Boxplot visualization functions ready


In [7]:
# Create aggregation level dropdown
aggregation_dropdown = widgets.Dropdown(
    options=[
        ('Individual Cities', 'individual_cities'),
        ('City Size Categories', 'city_size')
    ],
    value='individual_cities',
    description='Aggregation:',
    layout={'width': '250px'},
    style={'description_width': 'initial'}
)

# Create metric selection dropdown
metric_dropdown = widgets.Dropdown(
    options=[
        ('Vulnerability Ratio (%)', 'pixel_vulnerability_ratio'),
        ('Under-5 Population', 'pixel_under_5_pop'),
        ('Over-65 Population', 'pixel_over_65_pop'),
        ('Total Population', 'pixel_total_pop')
    ],
    value='pixel_vulnerability_ratio',
    description='Metric:',
    layout={'width': '250px'},
    style={'description_width': 'initial'}
)

# Create the visualization button
viz_button = widgets.Button(
    description='📊 Create Boxplots',
    button_style='info',
    layout={'width': '200px'},
    disabled=True
)

def on_viz_click(button):
    if pixel_results is not None:
        with output_area:
            print("\n" + "="*60)
            print("📊 CREATING BOXPLOT VISUALIZATIONS")
            print("="*60)
            
            aggregation = aggregation_dropdown.value
            metric = metric_dropdown.value
            
            # Get the display labels correctly
            agg_options_dict = {value: label for label, value in aggregation_dropdown.options}
            metric_options_dict = {value: label for label, value in metric_dropdown.options}
            
            agg_label = agg_options_dict[aggregation]
            metric_label = metric_options_dict[metric]
            
            print(f"🎯 Aggregation Level: {agg_label}")
            print(f"📈 Metric: {metric_label}")
            print(f"🌍 Country: {pixel_results['country_name'].iloc[0]}")
            print(f"🏙️ Cities: {pixel_results['city_name'].nunique()}")
            print(f"📍 Total Pixels: {len(pixel_results):,}")
            
            # Create the appropriate boxplot
            title_suffix = {
                'pixel_vulnerability_ratio': 'Heat Vulnerability Ratio',
                'pixel_under_5_pop': 'Under-5 Population (per pixel)',
                'pixel_over_65_pop': 'Over-65 Population (per pixel)',
                'pixel_total_pop': 'Total Population (per pixel)'
            }.get(metric, metric.replace('_', ' ').title())
            
            if aggregation == 'individual_cities':
                create_vulnerability_boxplot(pixel_results, metric, title_suffix)
                print("\n💡 Try switching to 'City Size Categories' to see patterns across different city sizes!")
            elif aggregation == 'city_size':
                create_city_size_boxplot(pixel_results, metric, title_suffix)
                print("\n💡 Switch back to 'Individual Cities' to see city-specific patterns!")
            
    else:
        print("❌ No data available - run extraction first")

viz_button.on_click(on_viz_click)

# Update the extract button to enable visualization
def updated_extract_click(button):
    global pixel_results
    
    # Run the original extraction
    on_extract_click(button)
    
    # Enable visualization button if extraction succeeded
    if pixel_results is not None:
        viz_button.disabled = False

# Clear existing callbacks and add new one
extract_button._click_handlers.callbacks.clear()
extract_button.on_click(updated_extract_click)

print('✅ Enhanced visualization interface ready')
print('📊 Features:')
print('   • Individual Cities: Compare distributions across specific cities')
print('   • City Size Categories: Analyze patterns by city size (Under 500K, 500K-1M, 1M-5M, 5M-10M, Over 10M)')
print('   • Multiple Metrics: Vulnerability ratio, age-specific populations, total population per pixel')

✅ Enhanced visualization interface ready
📊 Features:
   • Individual Cities: Compare distributions across specific cities
   • City Size Categories: Analyze patterns by city size (Under 500K, 500K-1M, 1M-5M, 5M-10M, Over 10M)
   • Multiple Metrics: Vulnerability ratio, age-specific populations, total population per pixel


In [8]:
# Display the complete interface
display(widgets.VBox([
    widgets.HTML('<h3>🔬 Pixel-Level Heat Vulnerability Analysis</h3>'),
    widgets.HTML('''
    <p><strong>Extract individual WorldPop pixel data for all cities in a selected country.</strong></p>
    <p><strong>Visualization Options:</strong></p>
    <ul>
        <li><strong>Individual Cities:</strong> Compare distributions across specific cities</li>
        <li><strong>City Size Categories:</strong> Analyze patterns by city size (Under 500K, 500K-1M, 1M-5M, 5M-10M, Over 10M)</li>
        <li><strong>Multiple Metrics:</strong> Vulnerability ratio, age-specific populations, total population per pixel</li>
    </ul>
    '''),
    
    widgets.HTML('<h4>📊 Data Extraction Configuration</h4>'),
    widgets.HBox([country_dropdown]),
    widgets.HBox([max_cities_slider, max_pixels_slider]),
    
    widgets.HTML('<h4>📈 Visualization Configuration</h4>'),
    widgets.HBox([aggregation_dropdown, metric_dropdown]),
    
    widgets.HTML('<h4>🎮 Controls</h4>'),
    widgets.HBox([extract_button, viz_button, export_button]),
    
    output_area
]))

print('\n🔬 Complete pixel-level analysis interface ready!')
print('📊 Enhanced Features:')
print('   • City Size Analysis: Compare vulnerability patterns across city size categories')
print('   • Multiple Metrics: Analyze different population measures')
print('   • Flexible Aggregation: Switch between individual cities and size-based grouping')
print('\n💡 Workflow: Extract Data → Select Aggregation & Metric → Create Boxplots → Export CSV')
print('\n🏙️ City Size Categories:')
print('   • Under 500K: Small cities and towns')
print('   • 500K - 1M: Medium cities') 
print('   • 1M - 5M: Large cities')
print('   • 5M - 10M: Major metropolitan areas')
print('   • Over 10M: Megacities')
print('\n🔧 Quick Test Settings: Brazil, 5 cities, 2,000 pixels/city')


🔬 Complete pixel-level analysis interface ready!
📊 Enhanced Features:
   • City Size Analysis: Compare vulnerability patterns across city size categories
   • Multiple Metrics: Analyze different population measures
   • Flexible Aggregation: Switch between individual cities and size-based grouping

💡 Workflow: Extract Data → Select Aggregation & Metric → Create Boxplots → Export CSV

🏙️ City Size Categories:
   • Under 500K: Small cities and towns
   • 500K - 1M: Medium cities
   • 1M - 5M: Large cities
   • 5M - 10M: Major metropolitan areas
   • Over 10M: Megacities

🔧 Quick Test Settings: Brazil, 5 cities, 2,000 pixels/city
